In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [6]:
df=pd.read_csv('real_estate_delhi_ncr_arranged.csv')

In [8]:
df

,City,SubLocation,Name,Price,Rate_per_sqft,Bedroom,Status,Transaction,Carpet_area_sqft,Total_area
0,Delhi,AG1 Pocket Vikaspuri,"2 BHK Flat for Sale in AG1 Pocket Vikaspuri, V...",12500000,13228.0,2,Ready to Move,New Property,NaN,944.965225
1,Delhi,Aastha Kunj Apartments,"3 BHK Flat for Sale in Aastha Kunj Apartments,...",13000000,7449.0,3,Ready to Move,New Property,2600.0,1745.200698
2,Delhi,Adarsh Apartment,"2 BHK Flat for Sale in Adarsh Apartment, Rohin...",17000000,6222.0,2,Ready to Move,Resale,2620.0,2732.240437
3,Delhi,Adarsh Nagar Extension,"2 BHK Flat for Sale in Adarsh Nagar Extension,...",10500000,11667.0,2,Ready to Move,Resale,525.0,899.974286
4,Delhi,Aditi Apartment,"3 BHK Flat for Sale in Aditi Apartment, Indrap...",17500000,10606.0,3,Ready to Move,Resale,NaN,1650.009429
...,...,...,...,...,...,...,...,...,...,...
2340,Faridabad,Tirka Colony,"3 BHK House for Sale in Tirka Colony, Mathura ...",4800000,10000.0,3,Ready to Move,Resale,NaN,480.000000
2341,Faridabad,Urban One,"2 BHK Flat for Sale in Urban One, Dayal Bagh C...",3000000,6422.0,2,Ready to Move,Resale,750.0,467.144192
2342,Faridabad,Woodbury Tower,"3 BHK Flat for Sale in Woodbury Tower, Suraj K...",12500000,7645.0,3,Ready to Move,Resale,1150.0,1635.055592
2343,Faridabad,sai park Apartment sector 87,2 BHK Flat for Sale in sai park Apartment sect...,4600000,4875.0,2,Ready to Move,Resale,NaN,943.589744


In [10]:
df.duplicated().sum()

0

In [12]:
df.isnull().sum().sum()

1452

In [14]:
df.isnull().sum()

City                   0
SubLocation            0
Name                   0
Price                  0
Rate_per_sqft         88
Bedroom                0
Status               128
Transaction            1
Carpet_area_sqft    1147
Total_area            88
dtype: int64

In [24]:
# Dropping the carpet_are_sqft
df.drop('Carpet_area_sqft',axis=1,inplace=True)

In [26]:
df.Transaction.value_counts()

Transaction
Resale          1873
New Property     467
Other              4
Name: count, dtype: int64

In [28]:
df=df[df.Transaction=='Resale']

In [30]:
df.isnull().sum()

City               0
SubLocation        0
Name               0
Price              0
Rate_per_sqft     72
Bedroom            0
Status           102
Transaction        0
Total_area        72
dtype: int64

In [32]:
df.to_csv('Resale.csv')